In [108]:
from bs4 import BeautifulSoup
import requests
from cleantext import clean
import re

In [109]:

# url = "https://eng.yidaiyilu.gov.cn/list/w/know_about"
def get_links_from_home(url):
    req = requests.get(url)
    soup = BeautifulSoup(req.text, "html.parser")
    # setup
    all_links = soup.ul
    # all the links in the <ul> tag on the yidaiyilu site. This contains all the sources they link in their online list. 
    all_links = str(all_links)
    # turning into a string object instead of a bs4 object. 
    all_content_list = all_links.split(" ")
    # splitting based on spaces, this is just to make it into a legible list. 
    links_list = []
    i = 0
    for value in all_content_list: 
        # looping through the list, building a new list with only the links. 
        if "href" in value: 
            links_list.append(value)
    links_cleaned_list = []
    for value in links_list: 
        # cleaning the links-- if it doesn't have a full https in front of it already adding it, removing the html href tag. 
        if "https://eng.yidaiyilu.gov.cn" not in value:
            full_link = "https://eng.yidaiyilu.gov.cn"+value[5:]
            links_cleaned_list.append(full_link.replace('"', ''))
        if value[-1] != "l":
            value+"l"
        else:
            links_cleaned_list.append(value[5:])
    print(links_cleaned_list)
    return(links_cleaned_list)

In [110]:
def clean_title(original_title, file_use):
  """
  helper to clean up the title for usage in creating the .txt file
  """
  list_title = list(str(original_title))
  starting_point = list_title.index(">")
  stopping_point = list_title.index("<", 59)
  output = ""
  for x in range(starting_point+10, stopping_point):
    output= output + list_title[x]
    x+=1
  if file_use:  
    output = output.replace(" ", "")
    output = output.replace('\n', "")
    final_output = ''.join(letter for letter in output if letter.isalnum())
    return str(final_output)
  else: 
      output = output.replace('\n', "")
      return str(output)


In [111]:
def get_source(all_text): 
  """
  Fixing up the Updated, Source fields
  all_text is all of text content from webpage. Should be a string.
  Use to find the Source, Updated, and Editor fields in the article.
  """
  for x in range(len(all_text)):
    if "Source:" in all_text[x]: return(all_text[x]+ " "+ all_text[x+1] + "\n")

def get_editor(all_text):
  """
  getting the editor name
  """
  for x in range(len(all_text)):
    if "Editor:" in all_text[x]: return(all_text[x]+" "+all_text[x+1] + "\n")

def get_updated(all_text):
  """
  getting the last updated timestamp
  """
  for x in range(len(all_text)):
    if "Updated:" in all_text[x]: return(all_text[x]+ " "+all_text[x+1]+"\n")


In [3]:
def paragraph_cleanup(article_content):
  """
  cleaning up the <p> content
  1. turn article_content into a list
  2. turn list contents into strings
  3. if not in there, write it into file, else don't.

  """
  # print(article_content)
  article_content_list = list(article_content)

  for content in article_content_list: 
    content = str(content)
    if("data" not in content):
      content = content.replace("<p>", "").replace("</p>", "")
      return(content + "\n")

In [112]:
# Getting the raw text data
def get_raw_data(link):
  """"
  Setup 
  """
  # f = open("demo.txt", "w")
  # link = "https://eng.yidaiyilu.gov.cn/p/0H647R18.html"
  link_req = requests.get(link).text.encode("utf8").decode("ascii", "ignore")
  page_soup = BeautifulSoup(link_req, "html.parser")
  article_content = page_soup.find_all("p")
  article_content = re.sub("<[^>]+>", '', str(article_content))
  article_content = article_content.replace("[", '')
  article_content = article_content.replace("]", '')
  # print(article_content)
  # everything with a <p> tag
  all_text = page_soup.find_all(string = True)
  # all the string content; useful for getting the date updated, source, editor
  title = page_soup.find_all("h1", string=True)
  filename = clean_title(title, True)+".txt"
  # getting a name for the file
  f = open(filename, "w")
  f.write(link + "\n")
  f.write(clean_title(title, False) + "\n")
  f.write(get_source(all_text))
  f.write(get_editor(all_text).strip()+"\n")
  f.write(get_updated(all_text))
  f.write(paragraph_cleanup(article_content)+"\n")
  f.write(article_content)
  # getting the title to write into the file
  

In [114]:
# for each link:
# 
# 1. create a new file (open())
# 2. write the text content into the file
# 3. close the file.

def mover():
  links = get_links_from_home("https://eng.yidaiyilu.gov.cn/list/w/know_about")
  for link in links:
    get_raw_data(link)
    
mover()

['https://eng.yidaiyilu.gov.cn/p/0H647R18.html', 'https://eng.yidaiyilu.gov.cn/p/0PPN0HMN.html', 'https://eng.yidaiyilu.gov.cn/p/0N4P7CF0.html', 'https://eng.yidaiyilu.gov.cn/p/006Q9GBH.html', 'https://eng.yidaiyilu.gov.cn/p/0CESAH1N.html', 'https://eng.yidaiyilu.gov.cn/p/041D0RIJ.html', 'https://eng.yidaiyilu.gov.cn/p/86739.html', 'https://eng.yidaiyilu.gov.cn/p/46076.html', 'https://eng.yidaiyilu.gov.cn/p/34863.html', 'https://eng.yidaiyilu.gov.cn/p/30277.html', 'https://eng.yidaiyilu.gov.cn/p/16639.html', 'https://eng.yidaiyilu.gov.cn/p/13754.html', 'https://eng.yidaiyilu.gov.cn/p/12731.html', 'https://eng.yidaiyilu.gov.cn/p/12479.html', 'https://eng.yidaiyilu.gov.cn/p/10477.html', 'https://eng.yidaiyilu.gov.cn/p/1084.html']
